In [ ]:
import pandas as pd
import numpy as np
import zipfile
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import json
import urllib.request
from urllib.error import HTTPError

file_path = 'file_path'
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
ApiKey='ApiKey'
UnitGroup='us'
ContentType="json"
Include="current"

def get_weather(idx, Location, StartDate):
    ApiQuery=BaseURL + Location
    ApiQuery+="/"+StartDate
    ApiQuery+="?"
    ApiQuery+="&unitGroup="+UnitGroup
    ApiQuery+="&contentType="+ContentType
    ApiQuery+="&include="+Include
    ApiQuery+="&key="+ApiKey
    while True:
        try:
          response = urllib.request.urlopen(ApiQuery)
        except HTTPError:
          print('##################################')
          print(idx, ApiQuery)
          continue
        break

    data = response.read()
    data = json.loads(data.decode('utf-8'))
    description = data['days'][0]['description']
    currentConditions = data['currentConditions']
    currentConditions.pop('stations', None)
    if idx%5000==0:
        print(idx, description)
    return {**currentConditions, **{'description': description}}

df = pd.read_csv(file_path)[['pickup_latitude', 'pickup_longitude', 'pickup_datetime']]
df['pickup_datetime'] = df['pickup_datetime'].astype('datetime64[ns]')
df['pickup_datetime_hour'] = pd.to_datetime(df['pickup_datetime'].dt.strftime('%Y-%m-%d %H'))
df = df[['pickup_latitude', 'pickup_longitude', 'pickup_datetime_hour']].values
weather = [get_weather(i, str(row[0])+','+str(row[1]), row[2].replace(" ", "T" )) for i, row in enumerate(df)]
weather = pd.DataFrame.from_records(weather)
df = pd.concat((df.reset_index(drop=True), weather), 1)
df.to_csv(file_path, index=False)